jd's python 3 testing/reference notebook  
(work in progress!)

In [1]:
import locale
import warnings
import json
import re
import sys

# Built-in types overview

|                  | type(x)    | x               | notes |
| ---------------- | ---------- | --------------- | ----- |
| numbers          | int        | `5`             | dynamically sized; includes all the overhead of any other object in Python (a pointer to its type, number of references...); at minimum, 28 bytes/int (`sys.getsizeof(12345)`); NumPy can avoid this overhead (as can the `array` module?)
|                  | float      | `5.0`           | see `sys.float_info` for max value and precision
|                  | complex    | `5 + 3j`        | implemented as two floats
| text sequence    | str        | `'xyz'`         | ordered, immutable, sequence of textual characters
| object sequences | tuple      | `('abc', 123)`  | ordered, immutable sequence of objects
|                  | list       | `['xyz', 890]`  | ordered, mutable sequence of objects
| binary sequences | bytes      | `b'Hello'`      | ordered, immutable sequence of bytes (ints 0-255)
|                  | bytearray  | `bytearray(5)`  | ordered, mutable sequence of bytes. Arg can be a value (specifying the array size, which will be zero-filled), an iterable of ints to copy, or existing binary `b'data'`. Alternatively, `bytearray.fromhex()` can read  hex data to create the array.
|                  | memoryview | `memoryview(bytes(5))`
| mappings         | dict       | `{'a': 1, 'b': 2}` | ordered (fwiw), mutable collection of key:value pairs. keys must be unique, hashable objects; values can be any object
| sets             | set        | `{1, 2, 3}`     | unordered, mutable collection of unique objs
|                  | frozenset  | `frozenset(`*iterable*`)` | unordered, immutable collection of unique objs
| generator        | range      | `range(1,8,2)`  | iterable generator for values; here: start 1, stop 8, step 2 -> (1,3,5,7)
| boolean type 	   | bool       | `True`          | https://docs.python.org/3/library/stdtypes.html#truth
| null object      | NoneType   | `None`          | there is one null object, "None" (note: automatically returned by functions that don't explicitly return a value)

- HASHABLES (any object which you can call `hash()` on)
    - Dictionary keys and set elements must be hashable (because dictionaries
      and sets are implemented using a hash table for lookup)
    - Built-in immutable objects (*e.g.*, `str`, `int`, `bool`, `bytes`) are
      generally 'hashable'
    - The exceptions are `frozenset` and `tuple`, which are hashable iff they
      contain only immutables
    - Mutable objects (*e.g.*, `list`, `dict`, `set`, `tuple`s containing e.g.,
      `list`s) are 'unhashable'.
- ITERABLES (objects which can return their members one at a time)
    - sequences (accessible with integer indices): text (`str`), object (`list`,
      `tuple`), and binary (`bytes`, `bytearray`, `memoryview`)
    - mappings (accessible with arbitrary, immutable keys): `dict`
    - sets: `set`, `frozenset`
    - generators: `range()`s, [generator functions](#generators),
      [generator expressions](#generator-expressions)
    - [file-type objects](#disk) created with `open()`
    - any other object with either
        1. an `__iter__()` method, which should return an iterator object which
           implements `__next__()` and raises `StopIteration` when no more
           elements are available
        2. a `__getitem__()` method which implements sequence semantics --
           which seems to mostly mean it has a `__len__()`, and its indicies
           run from `0` to `len()`, but the documentation is inconsistent and
           the code is obscure.
- SUBSCRIPTABLES:
    - An object is subscriptable if it implements `__getitem__()`.
    - Often iterables are subscriptable and vice versa, but not always:
        - Sets, e.g., are iterable, but not subscriptable:
        - `re.match` returns a "`Match`" object that is subscriptable but not
          iterable. See following code block.

Types reference: https://docs.python.org/3/library/stdtypes.html

In [13]:
# subscriptable but not iterable:
m = re.match(r"\w+ (\w+)", "Cats woofed, dogs meowed")
print('re.match returns', m)
print('Subscriptable: index "1" is the first parentheses:', m[1])
print('Explicit lookup with re.Match.__getitem__(1):', m.__getitem__(1))
try:
    for item in m:
        print(item)
except TypeError as err:
    print('Tried to iterate but got TypeError:', err)

# I believe it's not iterable because re.Match doesn't define __len__()
# (which I understand to be a requirement for using __getitem__ as an iterable)
try:
     print(len(m))
except TypeError as err:
     print('Tried to len() but got TypeError:', err)

re.match returns <re.Match object; span=(0, 11), match='Cats woofed'>
Subscriptable: index "1" is the first parentheses: woofed
Explicit lookup with re.Match.__getitem__(1): woofed
Tried to iterate but got TypeError: 're.Match' object is not iterable
Tried to len() but got TypeError: object of type 're.Match' has no len()


# Expressions

An [expression](https://docs.python.org/3/reference/expressions.html) is a
syntactic entity which evaluates to ('yields'/'returns') a value. 
* atomic expressions:
    * names: `name` (returns the value pointed to by name)
    * literals: `42`, `'foo'` (returns itself)
    * enclosures:
        * parenthesized expression: `(0)` (returns the expression inside the
          parentheses, `0`)
        * parenthesized tuples: empty pairs of parentheses `()` or parentheses
          containing at least one comma return tuples: `(0,)` -> `(0,)`
        * lists, sets, dicts (with contents either explicitly listed, or
          computed via a [comprehension](#list-comprehensions)) (return a new
          list/set/dict)
        * [generator expessions](#generator-expressions):
          `(x**2 for x in range(10))` (returns a new generator object)
        * `yield` expressions in generator functions
* primary expressions: ("the most tightly bound operations of the language")
    * attribute refereces: `name.attribute`
    * subscription: `container_name[subscript1, subscript2 ...]`
    * slicings: `sequence_name[index1, index2...]`,
      `sequence_name[start:stop:stride]`
    * calls: `callable_name(arg1, arg2, arg3='...')` (functions, built-ins,
      methods, classes)
* unary/binary arithmetic/bitwise operator expressions: `1 + 2` or `~bytes` or
  `"string" + "addenda"`. See [Operators](#Operators) below.
* comparisons and membership tests: `a < b` or `c not in d`. Yield `True` or
  `False`. See [Comparison operators](#comparison-operators) below.
* boolean negation expressions: `not x` (returns `True` if x is false, `False`
  otherwise)
* boolean conjuction expressions: `x and y` (returns `x` if x is false, `y`
  otherwise)
* boolean disjunction expressions: `x or y` (returns `x` if x is true, `y`
  otherwise)
* assignment expressions: whereas assignment (`x = y`) is a statement that
    yields no value, assignment expressions using the "walrus" operator `:=`
    both yield and assign an expression (`x := y` returns `y`, in addition to
    assigning it)
* conditional expressions (aka ternary operator) `x if condition else y`
  (returns either `x` or `y`)
* lambda expressions: `lambda x: x**2` (returns a function object)

Note: neither `and` nor `or` restrict the value and type they return to
False and True, but rather return the last evaluated argument. This is
sometimes useful, e.g., if `s` is a string that should be replaced by a default
value if it is empty, the expression `s or 'foo'` yields the desired value.

(Skipped: power expressions, await expressions in asynchronous coroutine
functions)

In [2]:
def testgen():
    yield None

print(testgen())
print(x for x in range(0))

<generator object testgen at 0x10f4bfcc0>
<generator object <genexpr> at 0x10f6fe670>


# Operators

## Assignment operators

In [3]:
# assignment (plus tuple unpacking, implicit grouping, iterable unpacking)
a, b, c = 1, 2, 3       # a == int(1)
foo = 1, 2, 3           # foo == tuple(1, 2, 3)
*foo, bar = 1, 2, 3     # bar == list(2, 3)
d, e, f, *almost_all_the_rest, g = range(1,10)
print(almost_all_the_rest)  # [4, 5, 6, 7, 8]

# augmented assignment (plus type coercion)
c += 2;     print(str(c).ljust(3), type(c))  # 5
c -= 1;     print(str(c).ljust(3), type(c))  # 4
c *= 2;     print(str(c).ljust(3), type(c))  # 8
c /= 2;     print(str(c).ljust(3), type(c))  # 4.0 - type coercion
c = int(c); print(str(c).ljust(3), type(c))  # 4
c //= 3;    print(str(c).ljust(3), type(c))  # 1 - floor division, no type coercion

'''
full list, from parser:
augassign: ('+=' | '-=' | '*=' | '@=' | '/=' | '%=' | '&=' | '|=' | '^=' |
            '<<=' | '>>=' | '**=' | '//=')
'''; #semicolon avoids printing value of final statement

[4, 5, 6, 7, 8]
5   <class 'int'>
4   <class 'int'>
8   <class 'int'>
4.0 <class 'float'>
4   <class 'int'>
1   <class 'int'>


## Walrus operator

Assignment *expressions* use the walrus `:=` operator to both yield and assign
the value of an expression.

In [4]:
if (match := re.search('f(o+)b', 'foooooooobar')):
    print(match.group(1))

# is the same as:

match = re.search('f(o+)b', 'foooooooobar')
if match:
    print(match.group(1))

# especially useful in a long if .. elif chain

oooooooo
oooooooo


## Mathematical and bitwise operators
from cpython/Grammar/python.gram

In [5]:
a = b = 1

# Binary mathematical operators
a + b  #Add
a - b  #Sub
a * b  #Mult
a / b  #Div
a // b #FloorDiv
a ** b #Power
a % b  #Mod
#c @ d #matrix multiply (no builtin types have __matmul__ method, cf. NumPy)

# Binary bitwise operators
a | b  # bitwise OR
a ^ b  # bitwise XOR
a & b  # bitwise AND
a << b # left shift
a >> b # right shift

# Unary operators
print(+a) # unary Add
print(-a) # unary Sub
print(~a) # unary bitwise NOT (invert)
print(bin(a), bin(~a))

1
-1
-2
0b1 -0b10


## Comparison operators

These all yield `True` or `False`, unless the corresponding dunder method for
the object has been changed to return something fancier.

In [6]:
a = b = ''

# value comparisons
a < b   # a.__lt__(b)
a > b   # a.__gt__(b)
a <= b  # a.__le__(b)
a >= b  # a.__ge__(b)
a == b  # a.__eq__(b)
a != b  # a.__ne__(b)

# identity comparisons
a is b      # these use id() to test if a and b are the same object
a is not b  #   https://docs.python.org/3/library/functions.html#id

# membership tests
a in b      # a.__contains__(b), falling back to __iter__(), then __getitem__()
a not in b; #   https://docs.python.org/3/reference/expressions.html#comparisons


## Coda

Quotations not otherwise cited or linked are from the [Python 3
documentation](https://docs.python.org/3/), Copyright 2001-2023, Python
Software Foundation